In [4]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
import os

print(os.getcwd())

import sys
from pathlib import Path

/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025/TB


In [5]:
Path().resolve().parent

PosixPath('/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025')

In [10]:
# Add utils to path
ROOT_PATH = Path().resolve().parent
print(f"{ROOT_PATH=}")
DATA_PATH = ROOT_PATH / "for_participants" / "data_parquet"
assert DATA_PATH.exists()
sys.path.append(str(ROOT_PATH))
print(f"{DATA_PATH=}")

# ASSIGNMENT_DIR = Path().resolve().parent
# HANDIN_DIR = ASSIGNMENT_DIR / 'handin'
OUTPUT_DIR = ROOT_PATH / 'output'

ROOT_PATH=PosixPath('/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025')
DATA_PATH=PosixPath('/Users/tymoteuszbarcinski/DTU/WEC2025/wec2025/for_participants/data_parquet')


# Buildings

In [ ]:
df_buildings = pd.read_parquet(DATA_PATH / 'buildings.parquet')
print(df_buildings.shape)

In [14]:
df_buildings.head()

,h3res13,building_id,building_function,building_floors,building_flats,building_area
0,8d1f53cd3299dbf,3303EA7E-6861-33C1-E053-CA2BA8C02B7D,Agricultural Building,1.0,0,245.66
1,8d1f53cd3299d3f,3303EA7E-6861-33C1-E053-CA2BA8C02B7D,Agricultural Building,1.0,0,245.66
2,8d1f53cd3299cbf,3303EA7E-6861-33C1-E053-CA2BA8C02B7D,Agricultural Building,1.0,0,245.66
3,8d1f53cd329d6ff,3303EA7E-6861-33C1-E053-CA2BA8C02B7D,Agricultural Building,1.0,0,245.66
4,8d1f53cd329837f,3303EA7E-6861-33C1-E053-CA2BA8C02B7D,Agricultural Building,1.0,0,245.66


In [20]:
df_buildings['building_function'].value_counts().sort_values(ascending=False)

building_function
Single-Family Residence           418099
Multi-Family Residence            371381
Office Building                   115619
Retail and Service Building        97803
Agricultural Building              97422
Silo or Warehouse                  87941
Industrial Building                62074
Schools and Research Institute     54779
Car Park                           36810
Hotel                              18213
Apartment                          18062
Hospitals and Medical Facility     15025
Collected Dwelling Unit            13269
Railway and Terminal Building       7619
Cultural Facility                   6065
Place of Worship                    5624
Cultural Public Facility            5395
Museums and Libraries               5082
Non-Residential Building            4016
Heritage Building                    316
Tourist Accommodation Building       256
Name: count, dtype: int64

# Demography

In [21]:
df_dem = pd.read_parquet(DATA_PATH / 'demography.parquet')
print(df_dem.shape)

(109678, 8)


In [22]:
df_dem

,h3res13,total,female0018,female1964,female6599,male0018,male1964,male6599
0,8d1f5224000837f,6.0,1.0,3.0,1.0,0.0,1.0,0.0
1,8d1f522400084bf,1.0,0.0,0.0,0.0,0.0,0.0,1.0
2,8d1f522400088ff,6.0,1.0,2.0,0.0,1.0,1.0,1.0
3,8d1f5224000943f,4.0,0.0,1.0,1.0,0.0,2.0,0.0
4,8d1f5224000a6ff,4.0,0.0,1.0,0.0,0.0,2.0,1.0
...,...,...,...,...,...,...,...,...
109673,8d1f53d9eda12ff,4.0,1.0,2.0,0.0,1.0,0.0,0.0
109674,8d1f53d9eda987f,0.0,0.0,0.0,0.0,0.0,0.0,0.0
109675,8d1f53d9edab47f,6.0,0.0,4.0,0.0,0.0,2.0,0.0
109676,8d1f53d9edadb7f,4.0,2.0,2.0,0.0,0.0,0.0,0.0


# 